In [ ]:
import requests 
import json
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
#Credentials
API_KEY = os.getenv("API_KEY")
ENDPOINT = os.getenv("ENDPOINT")
CLIENT = os.getenv("CLIENT")
COLLECTION_ID_OCR = os.getenv("COLLECTION_ID_OCR")
COLLECTION_ID_PREPROCESSING = os.getenv("COLLECTION_ID_PREPROCESSING")
COLLECTION_ID_NO_PREPROCESSING = os.getenv("COLLECTION_ID_NO_PREPROCESSING")


#Intializing
final_df_questions = []
final_df_expected_responses = []
final_df_expected_documents = []
ns_answers = []
documents_retrieved = []
processing_ocr_type = []


In [ ]:
def seek_api(input, api_key, url, collection_id, client_name): 
    
    url += '/seek'

    headers = {
            'accept': 'application/json',
            'apikey': api_key,
            'Content-Type': 'application/json'
            }
    
    payload = {
        'question': f'{input}',
        'options': {
            'filter': collection_id,
            'company': client_name
        }
    }
    
    response = requests.post(url, headers=headers, json=payload)
    return response

def generate_answers(preprocessing, ocr, collection_id, questions, sources, expected_responses):
    message = ""
    if ocr:
        message = "Using OCR"
    else:
        if preprocessing:
            message = "Using Pre-Processing, No OCR"
        else:
            message = "No Pre-Processing, No OCR"

    for i in range(0, len(questions)):
        try:
            response = seek_api(questions[i], API_KEY, ENDPOINT, collection_id, CLIENT)
            json = response.json()
            answer = json['answer']
            document_retrieved = json['document']
            
            ns_answers.append(answer)
            documents_retrieved.append(document_retrieved)
            final_df_questions.append(questions[i])
            final_df_expected_documents.append(sources[i])
            final_df_expected_responses.append(expected_responses[i])
            processing_ocr_type.append(message) 
        except:
            ns_answers.append("NS API ERROR")
            documents_retrieved.append("NS API ERROR")
            final_df_questions.append(questions[i])
            final_df_expected_documents.append(sources[i])
            final_df_expected_responses.append(expected_responses[i])        
            processing_ocr_type.append(message)

In [ ]:
# Fill in below with your questions, expected sources, and expected responses
questions = []

sources = [] 

expected_responses = []

# Pre-Processing, No OCR

In [ ]:
generate_answers(True, False, COLLECTION_ID_PREPROCESSING, questions, sources, expected_responses)

# No Pre-Processing, No OCR

In [ ]:
generate_answers(False, False, COLLECTION_ID_NO_PREPROCESSING, questions, sources, expected_responses)

# Using OCR

In [ ]:
generate_answers(False, True, COLLECTION_ID_OCR, questions, sources, expected_responses)

In [ ]:
dict={'Question Asked': final_df_questions, 
      'NS Response': ns_answers, 
      'Expected Response': final_df_expected_responses, 
      'NS Document Retrieved': documents_retrieved, 
      'Expected Source Doc': final_df_expected_documents, 
      'processing_ocr_type': processing_ocr_type
      }
df = pd.DataFrame(dict)

In [ ]:
df

In [ ]:
for elem in [final_df_questions, ns_answers, final_df_expected_responses, documents_retrieved,final_df_expected_documents, processing_ocr_type]:
    print(len(elem))

In [ ]:
df.head()

In [ ]:
df.to_csv('~/Desktop/testing.csv', index=False)